In [1]:
import pandas as pd
import numpy as np
import math
import random
pd.set_option("display.max_rows", None)

In [18]:
data = pd.read_csv("Arrival and Departure Rates.csv")
data.head()

,Time Period,Lot ID,Lot Name,Arrival Rate,Departure Rate
0,7-8,1,Rams,90,6
1,8-9,1,Rams,70,10
2,9-10,1,Rams,60,10
3,10-11,1,Rams,50,10
4,11-12,1,Rams,20,15


In [19]:
data.columns

Index(['Time Period', 'Lot ID', 'Lot Name', 'Arrival Rate', 'Departure Rate'], dtype='object')

In [20]:
def compute_sum(A, S):
  total = 0
  for i in range(S+1):
    total += (A**i) / math.factorial(i)
  return total

def B(row):
  A = row['Arrival Rate']
  S = row['Departure Rate']
  prob = ((A**S)/math.factorial(S))/compute_sum(A,S)
  return prob

# def B(row):
#     A = row['Arrival Rate']
#     S = row['Departure Rate']
#     prob = ((A**S)/math.factorial(S))/compute_sum(A,S)
#     return prob

In [21]:
# compute blocking probabilities for each lot
data['Blocking Probability'] = data.apply(B, axis=1)
data.head()

,Time Period,Lot ID,Lot Name,Arrival Rate,Departure Rate,Blocking Probability
0,7-8,1,Rams,90,6,0.934108
1,8-9,1,Rams,70,10,0.859437
2,9-10,1,Rams,60,10,0.836518
3,10-11,1,Rams,50,10,0.804716
4,11-12,1,Rams,20,15,0.329997


In [27]:
# now using the blocking probability for each of these, simulate a weighted coin toss with 0 if the lot is blocked and 1 if it is not. create 1000 runs for each lot and time period
data['Simulation'] = data.apply(lambda x: np.random.choice([0,1], p=[x['Blocking Probability'], 1-x['Blocking Probability']], size=1000), axis=1)
data.head()

,Time Period,Lot ID,Lot Name,Arrival Rate,Departure Rate,Blocking Probability,Simulation
0,7-8,1,Rams,90,6,0.934108,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,8-9,1,Rams,70,10,0.859437,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
2,9-10,1,Rams,60,10,0.836518,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."
3,10-11,1,Rams,50,10,0.804716,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,11-12,1,Rams,20,15,0.329997,"[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, ..."


In [31]:
# Convert the lists in 'Simulation' to a DataFrame
simulation_df = pd.DataFrame(data['Simulation'].tolist())

# Add 'Lot Name' and 'Time Period' to the new DataFrame
simulation_df['Lot'] = data['Lot Name'] + " " + data['Time Period']

# Set 'Lot' as the index
simulation_df.set_index('Lot', inplace=True)

# Transpose the DataFrame
final_data = simulation_df.transpose()

final_data.head()


Lot,Rams 7-8,Rams 8-9,Rams 9-10,Rams 10-11,Rams 11-12,Rams 12-1,Rams 1-2,Rams 2-3,Rams 3-4,Rams 4-5,...,Davis 7-8,Davis 8-9,Davis 9-10,Davis 10-11,Davis 11-12,Davis 12-1,Davis 1-2,Davis 2-3,Davis 3-4,Davis 4-5
0,0,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
1,1,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
2,0,0,0,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
3,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1
4,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,1


In [32]:
# export this data to a csv called input_data.csv
final_data.to_csv('input_data.csv', index=False)